In [7]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import sklearn
import imblearn
import pickle
from joblib import load, dump

In [2]:
with open("../../models/features/dataset_ids_before_balancing.pkl", "rb") as f:
    lc_ids = pickle.load(f)

In [11]:
features = {}
for subset in ['train', 'validation', 'test']:
    tmp = []
    for lc_id in lc_ids[subset]:
        with open("features/fats"+str(int(lc_id))+".pkl", "rb") as f:
            tmp.append(load(f))
    features[subset] = pd.concat(tmp, axis=0)

In [ ]:
parameters = {'criterion':('gini', 'entropy'), 
              'n_estimators':[50, 500, 5000], 
              'max_depth': [None, 10] } 

rf = BalancedRandomForestClassifier() 
cv = GridSearchCV(rf, parameters, cv=5, verbose=2, n_jobs=8, scoring='f1_macro') 
cv.fit(features.loc[train_idx].fillna(0).values, df_meta.loc[train_idx]["label"]) 
print(cv.best_params_)

In [ ]:
print("Compressing best model") 

with open("best_rf_balanced.pkl.xz", "wb") as f: 
    dump(cv.best_estimator_, f, compress=('xz', 9)) 